In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")
test_df_backup = test_df.copy()

In [ ]:
train_df.describe()

In [ ]:
train_df.head()

In [ ]:
#The features Paseesnger ID, Name and ticket are not major factors for survival
#so lets drop them
train_df = train_df.drop(["PassengerId"], axis = 1)
train_df = train_df.drop(["Name"], axis = 1)
train_df = train_df.drop(["Ticket"], axis = 1)

In [ ]:
train_df.isnull().sum()

In [ ]:
#too many values (77%) are missing for the column, so lets drop it
train_df = train_df.drop(["Cabin"], axis = 1)

#Scope for improvement 1: now age is filled with just the mean, instead,
#for age of passenger lets fill in ages with random numbers within one standard deviation of mean
#as this region includes 68% of the values and also doesnt put in small / large values
age_mean = train_df["Age"].mean()
train_df["Age"].fillna(age_mean, inplace = True)

#lets fill in embarked with 'S' as most people embarked in Southampton
train_df["Embarked"].fillna('S', inplace = True)


In [ ]:
train_df.head()

In [ ]:
# converting data into numeric

#Sex
Sex_dict = {"male":1, "female":0}
train_df["Sex"] = train_df["Sex"].map(Sex_dict)

#Embarked
train_df["Embarked_C"] = (train_df["Embarked"] == 'C')
train_df["Embarked_Q"] = (train_df["Embarked"] == 'Q')
train_df["Embarked_S"] = (train_df["Embarked"] == 'S')
train_df = train_df.drop(["Embarked"], axis = 1)

In [ ]:
train_df.head()

In [ ]:
#lets convert the age into a more useful form

train_df["Child"] = (train_df["Age"] <= 11)
train_df["Young"] = (train_df["Age"] > 11) & (train_df["Age"] <= 20)
train_df["Adult"] = (train_df["Age"] > 20) & (train_df["Age"] <= 35)
train_df["Mature"] = (train_df["Age"] > 35) & (train_df["Age"] <= 50)
train_df["Old"] = (train_df["Age"] > 50) 

train_df.head()

In [ ]:
#normalize the Fare and Age and store the skewing values:

age_mean = train_df["Age"].mean()
age_stddev = train_df["Age"].std()
train_df["Age"] = (train_df["Age"] - age_mean)/age_stddev

fare_mean = train_df["Fare"].mean()
fare_stddev = train_df["Fare"].std()
train_df["Fare"] = (train_df["Fare"] - fare_mean)/fare_stddev

In [ ]:
train_df.head()

In [ ]:
#do the same processes to the test dataframe

#drop the useless
test_df = test_df.drop(["PassengerId"], axis = 1)
test_df = test_df.drop(["Name"], axis = 1)
test_df = test_df.drop(["Ticket"], axis = 1)

#take care of nan
test_df = test_df.drop(["Cabin"], axis = 1)
test_df["Age"].fillna(age_mean, inplace = True)
test_df["Embarked"].fillna('S', inplace = True)
#test df has one value of fare nan
test_df["Fare"].fillna(test_df["Fare"].mean(), inplace = True)

#convert to numeric
#Sex
test_df["Sex"] = test_df["Sex"].map(Sex_dict)
#Embarked
test_df["Embarked_C"] = (test_df["Embarked"] == 'C')
test_df["Embarked_Q"] = (test_df["Embarked"] == 'Q')
test_df["Embarked_S"] = (test_df["Embarked"] == 'S')
test_df = test_df.drop(["Embarked"], axis = 1)

#convert age
test_df["Child"] = (test_df["Age"] <= 11)
test_df["Young"] = (test_df["Age"] > 11) & (test_df["Age"] <= 20)
test_df["Adult"] = (test_df["Age"] > 20) & (test_df["Age"] <= 35)
test_df["Mature"] = (test_df["Age"] > 35) & (test_df["Age"] <= 50)
test_df["Old"] = (test_df["Age"] > 50) 

#Normalize Age and fare using the same skewing constants
test_df["Age"] = (test_df["Age"] - age_mean)/age_stddev
test_df["Fare"] = (test_df["Fare"] - fare_mean)/fare_stddev

test_df.head()

In [ ]:
X = train_df.drop(["Survived"], axis = 1).values.astype('float32')
y = train_df["Survived"].values.astype('int32')

X_test = test_df.values.astype('float32')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size = 0.25)

In [ ]:
X_train

In [ ]:
import tensorflow as tf

In [ ]:
#initialize model

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(25, activation = 'relu', input_shape=(14,)),
    tf.keras.layers.Dense(5, activation = 'relu'),
    tf.keras.layers.Dense(5, activation = 'relu'),
    tf.keras.layers.Dense(1)
])


#predictions
y_pred = model(X_train[:1]).numpy()
y_pred

In [ ]:
adam_op = tf.keras.optimizers.Adam(
    learning_rate= 0.001 , beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')


model.compile(optimizer = adam_op,
             loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 1)

In [ ]:
dev_loss, dev_accuracy = model.evaluate(X_dev, y_dev, verbose = 2)

In [ ]:
# iteration over different NN sizes:

# size                train_loss    train_accuracy    dev_loss   dev_accuracy  dropout           inference
# 15 10 10 10 1 ..... 0.4592        0.8069            0.4386     0.7982        no overfitting    
# 15 10 1 ........... 0.4171        0.8204            0.4153     0.8161        no overfitting     
# 7 5 5 1 ........... 0.3919        0.8293            0.4239     0.8341        no overfitting     
# 20 5 1 ............ 0.4198        0.8278            0.4271     0.8072        no overfitting     
# 25 5 1 ............ 0.3811        0.8398            0.4522     0.8430        0.1 0             25 best for first layer    
# 30 5 1 ............ 0.3948        0.8308            0.4024     0.8341        0.1 0
# 25 5 5 1 .......... 0.3657        0.8473            0.4108     0.8430        0.1 0            

In [ ]:
#now lets train the model with best result on the entire dataset and calculate the submission


model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(25, activation = 'relu', input_shape=(14,)),
    tf.keras.layers.Dense(5, activation = 'relu'),
    tf.keras.layers.Dense(5, activation = 'relu'),
    tf.keras.layers.Dense(1)
])


# alpha 0.01
# define optimizer with alpha
adam_op = tf.keras.optimizers.Adam(
    learning_rate= 0.001 , beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')

model.compile(optimizer = adam_op,
             loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics = ['accuracy'])
#train 10 epochs
model.fit(X, y, epochs = 20)

# alpha 0.001
# define optimizer with alpha
adam_op = tf.keras.optimizers.Adam(
    learning_rate= 0.001 , beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')

model.compile(optimizer = adam_op,
             loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics = ['accuracy'])
#train 8 epochs
model.fit(X, y, epochs = 10)

In [ ]:
predictions = model(X_test).numpy()
predictions = 1/(1 + np.exp(-predictions))
binary_predictions = predictions > 0.5
print(predictions[1:10])

In [ ]:
print(np.shape(binary_predictions))

In [ ]:
submission = pd.DataFrame(test_df_backup["PassengerId"])
submission["Survived"] = binary_predictions.astype("int32")
submission

In [ ]:
submission.to_csv('submissions.csv')